# Tweet Creator

In [1]:
import openai
import os 
from langchain.llms import OpenAI
from langchain.chat_models import AzureChatOpenAI
from langchain.prompts import ChatPromptTemplate,SystemMessagePromptTemplate,AIMessagePromptTemplate,HumanMessagePromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field


In [2]:
openai.api_key = os.getenv('OPENAI_API_KEY')
openai.api_base= os.getenv('OPENAI_API_BASE')
openai.api_type= "azure"
openai.api_version = os.getenv('OPENAI_API_VERSION')
deployment:str=os.getenv('CHATGPT_MODEL')

llm = AzureChatOpenAI(
    deployment_name=deployment,
    model_name=deployment
    )


In [3]:
class My_List(BaseModel):
    name: str = Field(description="")
    list_items: list = Field(description="")

def list_creator(topic):
    My_List.name=f"Name of the list of the {topic}"
    My_List.list_items=f"A list of the {topic}"
    
    parser = PydanticOutputParser(pydantic_object=My_List)
    
    human_text = "{instruction}\n{format_instructions}"
    human_prompt = HumanMessagePromptTemplate.from_template(human_text)
    
    chat_prompt = ChatPromptTemplate.from_messages([human_prompt])

    prompt = chat_prompt.format_prompt(instruction=f"Create a list for {topic}",
    format_instructions=parser.get_format_instructions()).to_messages()
    
    reply = llm(prompt)
     
    return parser.parse(reply.content)

def tweet_creator(topic, tone):
    system_template = 'You are a social media expert.'
    system_message = SystemMessagePromptTemplate.from_template(system_template)
    
    human_template = "Write a tweet about {topic}.  The tone should be {tone}.  The tweet should be no longer than 240 characters."
    human_message = HumanMessagePromptTemplate.from_template(human_template)
    
    chat_prompt = ChatPromptTemplate.from_messages([system_message,human_message])
    prompt = chat_prompt.format_prompt(topic=topic,tone=tone).to_messages()
    reply = llm(prompt)

    return reply.content

    
topic = input("Please enter a topic for a list: ")
result_list = list_creator(topic)
print("------------")
for i, result in enumerate(result_list.list_items, start=1):
    print(f"{i}. {result}")
    
topic_number = int(input("Enter the topic number to make a tweet about it."))    
tone = input("Enter the tone for the tweet.")
tweet = tweet_creator(result_list.list_items[topic_number -1], tone)
print("------------")
print(f"The tweet: {tweet}")



------------
1. {'name': 'Cairo', 'country': 'Egypt'}
2. {'name': 'Lagos', 'country': 'Nigeria'}
3. {'name': 'Kinshasa', 'country': 'Democratic Republic of Congo'}
4. {'name': 'Alexandria', 'country': 'Egypt'}
5. {'name': 'Casablanca', 'country': 'Morocco'}
6. {'name': 'Abidjan', 'country': 'Ivory Coast'}
7. {'name': 'Khartoum', 'country': 'Sudan'}
8. {'name': 'Dar es Salaam', 'country': 'Tanzania'}
9. {'name': 'Nairobi', 'country': 'Kenya'}
10. {'name': 'Addis Ababa', 'country': 'Ethiopia'}
------------
The tweet: "Oh, how I miss the vibrant streets and rich culture of #Casablanca. My heart aches for the warmth of the Moroccan sun and the taste of traditional mint tea. #Morocco, I long to return to your welcoming embrace." 😔☀️🍵 #travelnostalgia #missingMorocco
